## Preprocess data

In [367]:
## Open data
import pandas as pd
data = pd.read_sas('data/synth_cancer_linked_file.sas7bdat')
features = pd.read_csv('data/features.csv')

In [368]:
# drop rows
features = pd.read_csv('data/features.csv')
to_keep = features.names.values[features.Keep.values == 1]
data = data.loc[(data.YEARDIAG_SYNTH.values >= 2010) & (data.YEARDIAG_SYNTH.values <= 2014)]
data = data.loc[(data.SURVINT_SYNTH.values < 99998)]
data = data.loc[data.CENSOR_SYNTH.values < 2]
data = data.loc[data.STAGE_SYNTH.values < 6]

In [369]:
# bin features
data.HCDD_SYNTH = data.HCDD_SYNTH.apply(lambda x : 1 if x <= 4 else 2)
data.IMMDER_SYNTH = data.IMMDER_SYNTH.apply(lambda x : 2 if x >= 2 else 1)

cancer_features = list(range(np.where(data.columns == 'CERVIX_SYNTH')[0][0],
                          data.shape[1]))

for cancer_feature in cancer_features:
    feature_name = data.columns[cancer_feature]
    data[feature_name] = data[feature_name].apply(lambda x : 2 if x >= 2 else 1)

In [370]:
# label features
socio_econ_factors = ['TREGION_SYNTH', 'AGE_IMM_REVISED_group_SYNTH', 'CITBIR_SYNTH', 
                      'COWD_SYNTH', 'DVISMIN_SYNTH', 'EFCNT_PP_REVISED_SYNTH', 
                      'FOL_SYNTH', 'FPTIM_SYNTH', 'HCDD_SYNTH', 'IMMDER_SYNTH', 
                      'KID_GROUP_SYNTH', 'MARST_SYNTH', 'RUINDFG_SYNTH', 'YRIM_GROUP_SYNTH', 
                      'D_LICORATIO_DA_BEF_SYNTH', 'AGEDIAG_SYNTH', 'PREGION_SYNTH', 'SEX_SYNTH']

categorical_features = ['TREGION_SYNTH', 'STAGE_SYNTH', 'CENSOR_SYNTH', 'S_DEAD_SYNTH',
                        'SURG_1_SYNTH', 'SURG_2_SYNTH', 'AGE_IMM_REVISED_group_SYNTH', 
                        'CITBIR_SYNTH', 'COWD_SYNTH', 'DVISMIN_SYNTH', 'FOL_SYNTH', 
                        'FPTIM_SYNTH', 'HCDD_SYNTH', 'IMMDER_SYNTH', 'MARST_SYNTH', 
                        'RUINDFG_SYNTH', 'YRIM_GROUP_SYNTH', 'PREGION_SYNTH', 'SEX_SYNTH']

def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3 

socio_econ_factors = intersection(socio_econ_factors, to_keep)
categorical_features = intersection(categorical_features, to_keep)

In [371]:
stage = data.STAGE_SYNTH.values
is_late = np.zeros(data.shape[0])
is_late[stage >= 3] = 1
n_days_survive = data.SURVINT_SYNTH.values
is_prostate = data.PROSTATE_SYNTH.values

In [372]:
# map to categorical
for feature in categorical_features:
    data[feature] = data[feature].astype('category')

In [360]:
data = pd.get_dummies(data[socio_econ_factors])

## Analysis on survival rate

In [322]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score

def between_ss(labels, n_days_survive):
    n_labels = np.unique(labels).shape[0]
    sum_ = 0
    mean = np.mean(n_days_survive)

    for i in range(n_labels):
        ni = sum(labels == i)
        cluster_mean = np.mean(n_days_survive[labels == i])
        sum_ = sum_ + ni*((mean - cluster_mean)**2)
        
    return sum_/(n_labels - 1)

def within_ss(labels, n_days_survive):
    n_labels = np.unique(labels).shape[0]
    sum_ = 0
    
    for i in range(n_labels):
        cluster_mean = np.mean(n_days_survive[labels == i])
        sum_ = sum_ + sum(np.square(n_days_survive[labels == i] - cluster_mean))
        
    return sum_/(labels.shape[0] - n_labels)

def get_f_stat(labels, n_days_survive):
    return between_ss(labels, n_days_survive)/within_ss(labels, n_days_survive)

def get_means(labels, n_days_survive):
    n_labels = np.unique(labels).shape[0]
    means = np.zeros(n_labels)
    
    for i in range(n_labels):
        means[i] = np.mean(n_days_survive[labels == i])
        
    return means
    
n_clusters = list(range(2, 11))
max_f_stat = -1

for n_cluster in n_clusters:
    model = KMeans(n_cluster, n_init=100).fit(data)
    labels = model.labels_
    f_stat = get_f_stat(labels, n_days_survive)
        
    if f_stat > max_f_stat:
        best_n_cluster = n_cluster
        max_f_stat = f_stat
        max_f_labels = labels
        max_f_model = model

In [323]:
means = get_means(max_f_labels, n_days_survive)
worst_cluster = np.where(means == min(means))[0][0]

In [324]:
features = ['EFCNT_PP_REVISED_SYNTH', 'D_LICORATIO_DA_BEF_SYNTH', 'AGEDIAG_SYNTH', 
            'TREGION_SYNTH', 'AGE_IMM_REVISED_group_SYNTH', 'DVISMIN_SYNTH', 
            'HCDD_SYNTH', 'IMMDER_SYNTH', 'MARST_SYNTH', 'RUINDFG_SYNTH', 
            'SEX_SYNTH']

for feature in features:
    feature_ind = []
    
    for i in range(len(data.columns)):
        if feature in data.columns[i]:
            feature_ind.append(i)
            
    if len(feature_ind) == 1:
        print(feature + ': ' +
              str(max_var_model.cluster_centers_[worst_cluster][feature_ind][0]))
    else:
        max_val = max(max_var_model.cluster_centers_[worst_cluster][feature_ind])
        print(feature + ': ' +
             str(np.where(max_var_model.cluster_centers_[worst_cluster][feature_ind] == max_val)[0][0] + 1))

EFCNT_PP_REVISED_SYNTH: 1.8609077598828732
D_LICORATIO_DA_BEF_SYNTH: 5.126207906295771
AGEDIAG_SYNTH: 86.02225475841942
TREGION_SYNTH: 3
AGE_IMM_REVISED_group_SYNTH: 14
DVISMIN_SYNTH: 13
HCDD_SYNTH: 1
IMMDER_SYNTH: 1
MARST_SYNTH: 2
RUINDFG_SYNTH: 2
SEX_SYNTH: 1


## Analysis on late stage diagnosis

In [2]:
data = data[socio_econ_factors]

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score

def get_props(labels, is_late):
    # computes proportion of is late for each
    n_labels = np.unique(labels).shape[0]
    props = np.zeros(n_labels)
    
    for i in range(n_labels):
        props[i] = sum(is_late[labels == i])/sum(labels == i)
        
    return props

def get_variance(labels, is_late):
    # computes proportion of is late for each label
    # and returns total variance of the proportions
    
    props = get_props(labels, is_late)
    
    return np.var(props)


n_clusters = list(range(2, 11))
best_score = 0
max_var = -1

for n_cluster in n_clusters:
    model = KMeans(n_cluster, n_init=100).fit(data)
    labels = model.labels_
    variance = get_variance(labels, is_late)
    
    if variance > max_var:
        best_n_cluster = n_cluster
        max_var = variance
        max_var_labels = labels
        max_var_model = model

In [232]:
def get_props(labels, is_late):
    # computes proportion of is late for each
    n_labels = np.unique(labels).shape[0]
    props = np.zeros(n_labels)
    
    for i in range(n_labels):
        props[i] = sum(is_late[labels == i])/sum(labels == i)
        
    return props

props = get_props(max_var_labels, is_late)
worst_cluster = np.where(props == max(props))[0][0]

In [268]:
features = ['EFCNT_PP_REVISED_SYNTH', 'D_LICORATIO_DA_BEF_SYNTH', 'AGEDIAG_SYNTH', 
            'TREGION_SYNTH', 'AGE_IMM_REVISED_group_SYNTH', 'DVISMIN_SYNTH', 
            'HCDD_SYNTH', 'IMMDER_SYNTH', 'MARST_SYNTH', 'RUINDFG_SYNTH', 
            'SEX_SYNTH']

for feature in features:
    feature_ind = []
    
    for i in range(len(data.columns)):
        if feature in data.columns[i]:
            feature_ind.append(i)
            
    if len(feature_ind) == 1:
        print(feature + ': ' +
              str(max_var_model.cluster_centers_[worst_cluster][feature_ind][0]))
    else:
        max_val = max(max_var_model.cluster_centers_[worst_cluster][feature_ind])
        print(feature + ': ' +
             str(np.where(max_var_model.cluster_centers_[worst_cluster][feature_ind] == max_val)[0][0] + 1))

EFCNT_PP_REVISED_SYNTH: 2.56345800122624
D_LICORATIO_DA_BEF_SYNTH: 6.345187001839372
AGEDIAG_SYNTH: 58.410177805028084
TREGION_SYNTH: 3
AGE_IMM_REVISED_group_SYNTH: 14
DVISMIN_SYNTH: 13
HCDD_SYNTH: 1
IMMDER_SYNTH: 1
MARST_SYNTH: 2
RUINDFG_SYNTH: 2
SEX_SYNTH: 2
